Load annotation_matrix
Load Variants
Annotate Variants with annotation_matrix
Create models
Train models
save data

In [ ]:
import numpy as np
import polars as pl
import sys, os
from src.ncboost_functions import ncboost_train

l_path = '/data-cbl/bcaron/NCBoost/ncboost2/'

In [ ]:
A = ['GerpN', 'priPhCons', 'mamPhCons', 'verPhCons', 'priPhyloP', 'mamPhyloP', 'verPhyloP', 'GerpRS', 'GerpRSpval',
      'GerpS', 'ZooPriPhyloP', 'ZooVerPhyloP', 'ZooRoCC', 'ZooUCE', 'Roulette-AR']
B = ['bStatistic',  'CDTS', 'mean_MAF', 'mean_MAF_afr', 'mean_MAF_ami', 'mean_MAF_amr', 'mean_MAF_asj', 'mean_MAF_eas',
 'mean_MAF_fin', 'mean_MAF_mid', 'mean_MAF_nfe', 'mean_MAF_sas']
C = ['slr_dnds', 'gene_age', 'pLI', 'zscore_mis', 'zscore_syn', 'loeuf', 'GDI', 'ncRVIS',
 'ncGERP', 'RVIS_percentile', 'pcGERP']
region_list = ['upstream', 'downstream', 'intronic', 'UTR5', 'UTR3', 'intergenic']
D = ['GC', 'CpG', 'SpliceAI'] + region_list

variables = ['chr', 'pos', 'ref', 'rsid', 'region', 'closest_gene_name', 'closest_gene_ENSG', 
             'label', 'partition', 'matching_index', 'CADD', 'ReMM']

In [ ]:
pathogenic =  pl.read_csv(source = f'{l_path}/data/training/pathogenic_for_training_spliceai.tsv', 
                   separator = '\t',
                   schema_overrides = {'chr' : str, 'ZooRoCC' : int, 'ZooUCE' : int},
                   null_values = 'NA'
                   )

common =  pl.read_csv(source = f'{l_path}/data/training/common_for_training_spliceai.tsv', 
                   separator = '\t',
                   schema_overrides = {'chr' : str, 'ZooRoCC' : int, 'ZooUCE' : int},
                   null_values = 'NA'
                   )

In [ ]:
model_name = f'ncboost_models'
save_path = f'models/{model_name}'
if not os.path.exists(save_path):
    os.makedirs(save_path)

features = A + B + C + D

data = pl.concat([common.select(variables + features), pathogenic.select(variables + features)], how = 'vertical')

model_dict, feature_importance, annotated_data = ncboost_train(data, features, save_path)

In [ ]:
annotated_data.write_csv(file = f'{save_path}/scored_data.tsv', 
                        separator = "\t", 
                        include_header = True,
                        null_value = 'NA'
                        )